In [1]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd

In [2]:
episodes = []
for sn in range(1,7):
    response = requests.get('https://www.imdb.com/title/tt2575988/episodes?season='+str(sn))
    html_soup = BeautifulSoup(response.text, 'html5lib')
    episode_containers = html_soup.find_all('div', class_='info')
    for episode in episode_containers:
        season = sn
        episodeNum = episode.meta['content']
        episodeTitle = episode.a['title']
        airdate = episode.find('div', class_ = "airdate").text.strip()
        rating = episode.find('span', class_ = "ipl-rating-star__rating").text
        totalVotes = episode.find('span', class_ = "ipl-rating-star__total-votes").text
        desc = episode.find('div', class_ = 'item_description').text.strip()
        episode_data = [season, episodeNum, episodeTitle, airdate, rating, totalVotes, desc]
        episodes.append(episode_data)
episodes = pd.DataFrame(episodes, columns= ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc'])
episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Minimum Viable Product,6 Apr. 2014,7.7,"(2,765)",Richard is a computer programmer. He has to ch...
1,1,2,The Cap Table,13 Apr. 2014,7.7,"(2,229)",Richard hires Jared to design a business plan....
2,1,3,Articles of Incorporation,20 Apr. 2014,8.0,"(2,145)",The guys find themselves in trouble with their...
3,1,4,Fiduciary Duties,27 Apr. 2014,7.7,"(1,996)",Richard drunkenly promises to make Erlich a bo...
4,1,5,Signaling Risk,4 May 2014,8.3,"(2,045)",Now that Pied Piper is the official company na...


Next, add a function to clean the data.
Date format will be dd/mm/yyyy
and total votes change from (0000) to 0000

In [3]:
def cleaning(votes):
    for r in ((',',''), ('(',''),(')','')):
        votes = votes.replace(*r)
    return votes

episodes['total_votes'] = episodes.total_votes.apply(cleaning).astype(int)
episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Minimum Viable Product,6 Apr. 2014,7.7,2765,Richard is a computer programmer. He has to ch...
1,1,2,The Cap Table,13 Apr. 2014,7.7,2229,Richard hires Jared to design a business plan....
2,1,3,Articles of Incorporation,20 Apr. 2014,8.0,2145,The guys find themselves in trouble with their...
3,1,4,Fiduciary Duties,27 Apr. 2014,7.7,1996,Richard drunkenly promises to make Erlich a bo...
4,1,5,Signaling Risk,4 May 2014,8.3,2045,Now that Pied Piper is the official company na...


In [4]:
episodes.to_csv('SiliconValley_IMDb.csv', index = False)